# Prepare database

In [1]:
import os

dir_name = 'C:/Users/windo/Documents/ravdess-dataset/audio_speech_actors_01-24'

# Initialize a list to hold the file names
all_files = []

# Traverse all subfolders and files using os.walk()
for root, _, file_names in os.walk(dir_name):
    for file_name in file_names:
        file_path = os.path.join(root, file_name)
        if os.path.isfile(file_path):  # Check if it's a file
            all_files.append(file_path)

all_files = sorted(all_files)

# # Display the collected file names
# print('List of all files in the folders:')
# for file in all_files:
#     print(file)

# Emotions in the RAVDESS dataset
emotions = {
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

# desired csv columns:
# label/name(actor number, take, etc) emotion(class) feature1 feature2...
# or class at the end?
len(all_files)

1440

# Downsample the database to 'neutral', 'happy' and 'sad'

In [2]:
wanted_classes = {'01', '03', '04'}

filtered_files = [filepath for filepath in all_files if filepath.split('\\')[-1].split('-')[2] in wanted_classes]

len(filtered_files)
# Print the filtered filepaths
# for file in filtered_files:
#     print(file)

480

In [3]:
import numpy as np

import sys
sys.path.append("..")
import plap

# Use plap for parameterization
# Newest with ASB, ASP and variance across ASF
fvector = plap.FeatureVector("asc","asc_var","ass","ass_var","asf","asf_mean","asf_var","asf_var_mean","aff","aff_var","lat","tc","sc","sc_var","hsc","hsd","hss","hsv","asb","asb_mean","asp","asp_mean","mfcc")
# fvector = plap.FeatureVector("asc","asc_var","ass","ass_var","asf","asf_mean","aff","aff_var","lat","tc","sc","sc_var","hsc","hsd","hss","hsv")
# fvector = plap.FeatureVector("asc","ass","asf","lat","tc","sc","hsc","hsd","hss","hsv")
# block_size_ms = block_size_samples/sr*1000 = 1024/48000*1000 ~= 21.34 ms
preprocessor = plap.Preprocessor(preemphasis_coeff=None, block_size=1024, window_type="hamming")

def extract_features(audio_path):
    plap.parameterize(audio_path=audio_path, fvector=fvector, preprocessor=preprocessor)
    return fvector.values

In [ ]:
import csv

# Define the CSV filename
# fvector = plap.FeatureVector("asc","ass","asf","lat","tc","sc","hsc","hsd","hss","hsv")
csv_filename = './python_csv/ravdess_newest.csv'

# Feature names compatibility taking into account vector features
feature_names_compat = [feature.upper() for feature in fvector.features]

# Define mapping for base features and their corresponding counts
feature_mapping = {
    'ASF': 24,
    'ASF_VAR': 24,
    'ASB': 20,
    'ASP': 21,
    'MFCC': 20
}

i = 0
while i < len(feature_names_compat):
    base_feature = feature_names_compat[i]
    
    if base_feature in feature_mapping:
        count = feature_mapping[base_feature]
        feature_names_compat[i] = f'{base_feature}1'  # Set the first feature name
        
        # Create the numbered features list
        numbered_features = [f'{base_feature}{k}' for k in range(2, count + 1)]
        
        # Insert numbered features right after the base feature
        feature_names_compat[i+1:i+1] = numbered_features
        
        # Skip over the newly inserted features
        i += len(numbered_features)
    
    i += 1

feature_names_compat
# Initialize the CSV file with headers
# headers = ['label', 'class'] + [f'{feature.upper()}' for feature in feature_names_compat]
# with open(csv_filename, mode='w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(headers)

In [5]:
import time
# Iterate over each audio file
initial_i = 0
# end_i = 150
i = initial_i
processed = 0
for filename in filtered_files[initial_i:]:
    if filename.endswith('.wav'):  # Ensure only audio files are processed
        
        start_time = time.time()
        # Extract label and class from the filename (customize this logic as needed)
        audio_filename = filename.split('\\')[-1]
        label = audio_filename.removesuffix('.wav')  # Use filename as label
        class_num = audio_filename.split('-')[2]
        class_name = emotions[class_num]
        
        print(f'Processing file {audio_filename}')

        # Extract 11 features from the audio file
        try:
            features = extract_features(filename)
            processed += 1
        except:
            print(f'Omitting file{i}')
            i += 1
            continue
        # features = np.zeros(11)

        # Prepare the row to be added to the CSV
        row = [label, class_name] + list(features)

        # Append the new row to the CSV file
        with open(csv_filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(row)
        print(f'Finished file{i} in {time.time() - start_time} s')
        i += 1

print("Feature extraction and CSV writing completed.")

Processing file 03-01-01-01-01-01-01.wav
Finished file0 in 2.5316946506500244 s
Processing file 03-01-01-01-01-02-01.wav
Finished file1 in 0.4896838665008545 s
Processing file 03-01-01-01-02-01-01.wav
Finished file2 in 0.5352723598480225 s
Processing file 03-01-01-01-02-02-01.wav
Finished file3 in 0.49024438858032227 s
Processing file 03-01-03-01-01-01-01.wav
Finished file4 in 0.48969101905822754 s
Processing file 03-01-03-01-01-02-01.wav
Finished file5 in 0.5421533584594727 s
Processing file 03-01-03-01-02-01-01.wav
Finished file6 in 0.487107515335083 s
Processing file 03-01-03-01-02-02-01.wav
Finished file7 in 0.5476574897766113 s
Processing file 03-01-03-02-01-01-01.wav
Finished file8 in 0.5895712375640869 s
Processing file 03-01-03-02-01-02-01.wav
Finished file9 in 0.6314435005187988 s
Processing file 03-01-03-02-02-01-01.wav
Finished file10 in 0.6296327114105225 s
Processing file 03-01-03-02-02-02-01.wav
Finished file11 in 0.6555471420288086 s
Processing file 03-01-04-01-01-01-01.

C:\Users\windo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=2
  warnings.warn(


Omitting file386
Processing file 03-01-03-01-02-02-20.wav
Finished file387 in 0.9841587543487549 s
Processing file 03-01-03-02-01-01-20.wav
Finished file388 in 0.6150784492492676 s
Processing file 03-01-03-02-01-02-20.wav
Finished file389 in 0.7573986053466797 s
Processing file 03-01-03-02-02-01-20.wav
Finished file390 in 0.5977268218994141 s
Processing file 03-01-03-02-02-02-20.wav
Finished file391 in 0.6909852027893066 s
Processing file 03-01-04-01-01-01-20.wav
Finished file392 in 0.7523267269134521 s
Processing file 03-01-04-01-01-02-20.wav
Finished file393 in 0.6096000671386719 s
Processing file 03-01-04-01-02-01-20.wav
Finished file394 in 0.6237590312957764 s
Processing file 03-01-04-01-02-02-20.wav
Finished file395 in 0.5504562854766846 s
Processing file 03-01-04-02-01-01-20.wav
Finished file396 in 0.6287500858306885 s
Processing file 03-01-04-02-01-02-20.wav
Finished file397 in 0.7862646579742432 s
Processing file 03-01-04-02-02-01-20.wav
Finished file398 in 0.7131326198577881 s

In [ ]:
processed